In [1]:
import numpy as np
import ananke as an
an.__version__

'0.1.0.dev1'

We define here some dummy input data. Py-ananke has a method to produce such data.

In [2]:
np.random.seed(0)
p = an.Ananke.make_dummy_particles_input()
p.keys()

dict_keys(['pos3', 'vel3', 'mass', 'age', 'feh', 'helium', 'carbon', 'nitrogen', 'oxygen', 'neon', 'magnesium', 'silicon', 'sulphur', 'calcium', 'alpha', 'parentid', 'dform', 'log10_NH'])

The input data must be formatted as a dictionary of equal-length arrays. The dictionary must have the following entries:
- key `pos3`: particle position coordinates in $kpc$ (shape Nx3)
- key `vel3`: particle velocity coordinates in $km.s^{-1}$ (shape Nx3)
- key `mass`: particle stellar mass in solar masses
- key `age`: particle log10 stellar age in years
- key `feh`: particle stellar metallicity \[Fe/H\] in dex relative to solar

Additionally, the following entries can optionally be added:
- key `parentid`: index to give to the parent particle
- key `id`: additional index to classify the parent particle
- key `log10_NH`: log10 hydrogen column densities between Observer position and particle in $cm^{-2}$ - must be provided to estimate extinctions
- key `dform`: particle formation distance
- keys `helium`, `carbon`, `nitrogen`, `oxygen`, `neon`, `magnesium`, `silicon`, `sulphur`, `calcium`: particle various chemical abundances \[X/H\]
- key `alpha`: particle alpha chemical abundances \[Mg/Fe\]

We can define here some parameters for Ananke such as
 - the observer position `observer`
 - the shell of particles to mask `rshell`
 - the sampling factor `fsample`
 - the photometric system of choise `photo_sys`
 - the CMD `cmd_magnames` and its box limits `cmd_box`

In [3]:
D = 200 # *units.kpc

observer = np.nan*np.ones(3)
while not np.linalg.norm(observer)<1:
    observer = 2*np.random.rand(3)-1

observer *= D/np.linalg.norm(observer)

rshell = [0, 2*D]

fsample = 0.01

photo_sys = 'padova/GAIADR2'

cmd_magnames = {'magnitude': 'Gmag',
                'color_minuend': 'G_BPmag',
                'color_subtrahend': 'G_RPmag'}

cmd_box = {
           'abs_mag_lim_lo': -1000,
           'abs_mag_lim_hi': 1000,
        #    'app_mag_lim_lo' : -1000,
           'app_mag_lim_hi': 30,
        #    'color_lim_lo' : -1000,
        #    'color_lim_hi' : 1000
           }

For more details regarding these parameters and more, you may consult the docstring associated to the class `Ananke`:

In [4]:
help(an.Ananke)

Help on class Ananke in module ananke.Ananke:

class Ananke(builtins.object)
 |  Ananke(particles, name, ngb=64, d_params={}, e_params={}, err_params={}, **kwargs) -> None
 |  
 |  Represents a single ananke pipeline.
 |  
 |  Methods defined here:
 |  
 |  __init__(self, particles, name, ngb=64, d_params={}, e_params={}, err_params={}, **kwargs) -> None
 |      Parameters
 |      ----------
 |      particles : dict
 |          A dictionary of same-length arrays representing particles
 |          data of a stellar population - see notes for formatting
 |      
 |      name : str
 |          Name for the pipeline
 |      
 |      ngb : int
 |          Number of neighbours to use in kernel density estimation
 |      
 |      d_params : dict
 |          Parameters to configure the kernel density estimation. Use
 |          class method display_density_docs to find what parameters can
 |          be defined
 |      
 |      e_params : dict
 |          Parameters to configure the extinction

Preparing now the pipeline with GAIA photometry

In [5]:
name = 'sim'
ananke = an.Ananke(p, name, fsample=fsample,
                   observer=observer, rshell=rshell,
                   photo_sys=photo_sys, cmd_magnames=cmd_magnames,
                   **cmd_box)

/home/athob/miniconda3/envs/main/lib/python3.10/site-packages/ananke/utils.py:21: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  self._record_of_all_used_keys = set()
/home/athob/miniconda3/envs/main/lib/python3.10/site-packages/ananke/utils.py:21: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  self._record_of_all_used_keys = set()


The method `run` runs the pipeline:

In [6]:
survey = ananke.run()

Dimensions = 3
Normalization constant of Kernel type 3: 0.596831
Reading ASCII format file: to_enbid 
Read 300000 records
Allocated 3.8147 MByte for particle storage.
Type = 1 Particles = 100000
Total particles = 100000
Allocated 22.1252 MByte for Binary-Tree and Entropy bins.

Scaling Co-ordinates as x[i]=x[i]/h[i] with h[i]->
1 1 1 

Starting to Build Tree .......
Particle Type = 1 First node number = 1
                  Last node number  = 199999
Total number of nodes = 199999
Treebuild time = 0.084609 s 

Density Calculation. Smoothing .....
Evaluated =   1 % Time Left = 0.632307 s of 0.638700 s Par no = 96805 Density = 4.540412e-02 
Evaluated =   2 % Time Left = 0.602645 s of 0.614950 s Par no = 55454 Density = 4.964532e-03 
Evaluated =   3 % Time Left = 0.621408 s of 0.640633 s Par no = 27717 Density = 1.827217e-01 
Evaluated =   4 % Time Left = 0.623538 s of 0.649525 s Par no = 42234 Density = 2.712376e-02 
Evaluated =   5 % Time Left = 0.614625 s of 0.646980 s Par no = 99785 De

nbody1/sim/

	1	1

path nbody1/sim/ sats 1 x[0] 1


Isochrone Grid Size:                (Age bins=71,Feh bins=34,Alpha bins=1)
Time Isochrone Reading              1.82129     
------------------------------
nbody1/sim/sim.ebf  Sat No=0
Particles=100000
Satellite Info
Particles=100000 Mass=5.49989e+08 0.379706
Total Stars=2280137 accepted=1614133 rejected=666004
-----------Done---------------
Total stars written                 1614133                 
30
File written-                       /home/athob/Software/repos/py-ananke/jupyter/survey.sim.ebf
Calculating magnitudes in GAIADR2 system................
initializing isochrone data
interpolating on isochrone tables
Appending spherical coordinates................
Total Time=                         4.21442     
Exported the following quantities to /home/athob/Software/repos/py-ananke/jupyter/survey.sim.h5
['age', 'alpha', 'calcium', 'carbon', 'dec', 'dform', 'dmod', 'feh', 'gaiadr2_g_bpmag', 'gaiadr2_g_rpmag', 'gaiadr2_gmag', 'glat', 'glon', 'grav', 'helium', 'lum', 'mact', 'magnesium', 

/home/athob/miniconda3/envs/main/lib/python3.10/site-packages/ananke/_default_error_model.py:37: RuntimeWarning: overflow encountered in square
  grvs = rpmag + 132.32 - 377.28*ggrp + 402.32*ggrp**2 - 190.97*ggrp**3 + 34.026*ggrp**4
/home/athob/miniconda3/envs/main/lib/python3.10/site-packages/ananke/_default_error_model.py:37: RuntimeWarning: overflow encountered in multiply
  grvs = rpmag + 132.32 - 377.28*ggrp + 402.32*ggrp**2 - 190.97*ggrp**3 + 34.026*ggrp**4
/home/athob/miniconda3/envs/main/lib/python3.10/site-packages/ananke/_default_error_model.py:37: RuntimeWarning: overflow encountered in power
  grvs = rpmag + 132.32 - 377.28*ggrp + 402.32*ggrp**2 - 190.97*ggrp**3 + 34.026*ggrp**4
/home/athob/miniconda3/envs/main/lib/python3.10/site-packages/ananke/_default_error_model.py:37: RuntimeWarning: invalid value encountered in subtract
  grvs = rpmag + 132.32 - 377.28*ggrp + 402.32*ggrp**2 - 190.97*ggrp**3 + 34.026*ggrp**4
/home/athob/miniconda3/envs/main/lib/python3.10/site-package

Exported the following quantities to /home/athob/Software/repos/py-ananke/jupyter/survey.sim.h5
['pi_Err', 'vr_Err', 'gaiadr2_gmag_Err', 'vr_Sig', 'dec_Err', 'dec_Sig', 'ra_Err', 'gaiadr2_g_rpmag_Err', 'ra_Sig', 'gaiadr2_g_bpmag_Sig', 'gaiadr2_g_rpmag_Sig', 'mudec_Sig', 'gaiadr2_g_bpmag_Err', 'mura_Sig', 'mudec_Err', 'mura_Err', 'gaiadr2_gmag_Sig', 'pi_Sig']
Overwritten the following quantities to /home/athob/Software/repos/py-ananke/jupyter/survey.sim.h5
('gaiadr2_gmag', 'gaiadr2_g_rpmag', 'gaiadr2_g_bpmag', 'pi', 'ra', 'dec', 'mura', 'mudec', 'vr')
Overwritten the following quantities to /home/athob/Software/repos/py-ananke/jupyter/survey.sim.h5
('glon', 'glat', 'mul', 'mub')


The output is saved as a `vaex` dataframe, with its columns organized in alphabetical order. These notably include:
- key `A_0` for the reference extinction which extinction coefficients are based on (at $\lambda = 550 \, nm$ in the case of Gaia DR2)
- key `A_{filter_name}` for the extinction value in each filter designated by `filter_name` (in this case, where `filter_name` is one of the 3 Gaia bands `gaia_g_bpmag`, `gaia_g_rpmag` \& `gaia_gmag`)
- key `E(B-V)` for the reddening index
- key `age` for the log10 stellar age in years
- key `alpha`, `calcium`, `carbon`, `helium`, `magnesium`, `neon`, `nitrogen`, `oxygen`, `silicon`, `sulphur` for the various chemical abundances as given as input
- key `dec`, `ra` for the astrometric declination and right ascension celestial coordinates in degrees
- key `dform` for the formation distance as given as input
- key `dmod` for the distance modulus
- key `feh` for the stellar metallicity \[Fe/H\] in dex relative to solar
- key `glat`, `glon` for the astrometric galactic latitude and longitude celestrial coordinates in degrees
- key `grav` for the log10 surface gravity in CGS units
- key `log10_NH` for the log10 hydrogen column density between Observer position and star in $cm^{-2}$
- key `lum` for the stellar luminosity in solar luminosities
- key `mact`, `mtip`, `mini` for respectively the current stellar mass, the mass of that same star at tip of giant branch for its given age \& metallicity and its stellar mass on zero-age main sequence, all in solar masses
- key `mub`, `mudec`, `mul`, `mura` for the astrometric proper motions, respectively in the direction of the galactic latitude, declination, galactic longitude and right ascension, all in milliarcseconds per year
- key `parentid` for the parent particle index as given as input
- key `partid` for the flag that identifies stars that are *not* central relatively to their parent particle
- key `pi` for the star parallax in milliarcseconds
- key `px`, `py`, `pz` for the star position cartesian coordinates in $kpc$ relative to the Observer's position
- key `rad` for the star distance to the Observer in $kpc$
- key `teff` for the star effective temperature in Kelvin
- key `vr` for the star astrometric radial velocity in $km.s^{-1}$
- key `vx`, `vy`, `vz` for the star velocity cartesian coordinates in $km.s^{-1}$ relative to the Observer's velocity

Additionally, astrometric and photometric quantities `X` all have associated columns identified as:
- key `X_Sig` for the standard error on the quantity `X`
- key `X_Err` for the actual drawn gaussian error on the quantity `X`

In [7]:
survey

#,A_0,A_gaiadr2_g_bpmag,A_gaiadr2_g_rpmag,A_gaiadr2_gmag,E(B-V),age,alpha,calcium,carbon,dec,dec_Err,dec_Sig,dform,dmod,feh,gaiadr2_g_bpmag,gaiadr2_g_bpmag_Err,gaiadr2_g_bpmag_Intrinsic,gaiadr2_g_bpmag_Sig,gaiadr2_g_rpmag,gaiadr2_g_rpmag_Err,gaiadr2_g_rpmag_Intrinsic,gaiadr2_g_rpmag_Sig,gaiadr2_gmag,gaiadr2_gmag_Err,gaiadr2_gmag_Intrinsic,gaiadr2_gmag_Sig,glat,glon,grav,helium,log10_NH,lum,mact,magnesium,mtip,mub,mudec,mudec_Err,mudec_Sig,mul,mura,mura_Err,mura_Sig,neon,nitrogen,oxygen,parentid,partid,pi,pi_Err,pi_Sig,px,py,pz,ra,ra_Err,ra_Sig,rad,silicon,smass,sulphur,teff,vr,vr_Err,vr_Sig,vx,vy,vz
0,0.7093970129666665,0.7125676091560126,0.4070167834122857,0.5278885380814727,0.2288377461182795,10.016325950622559,0.3380742371082306,-1.1830222606658936,-0.39430123567581177,nan,nan,nan,0.0,20.87123488088669,-0.8867433071136475,nan,nan,8.22037,nan,nan,nan,6.875088,nan,nan,nan,7.6075797,nan,nan,nan,56622.53,-0.7315406799316406,21.757467670308866,0.08502915,0.52380776,-0.5486690402030945,0.886202,nan,nan,nan,nan,nan,nan,nan,nan,-0.7326117157936096,-0.652482807636261,-0.6697278618812561,0,0,nan,nan,nan,-142.03525597960677,46.20841677042607,0.692591828190988,nan,nan,nan,149.36435788052748,-0.7563512325286865,0.5240775107307198,-1.0924330949783325,3.643452,nan,nan,nan,80.28685760498047,-54.7354850769043,62.91861343383789
1,0.42736475124343404,0.42914098540345996,0.24512763624360917,0.31782544112867933,0.1378595971753013,10.016325950622559,0.3380742371082306,-1.1830222606658936,-0.39430123567581177,nan,nan,nan,0.0,20.874669804041112,-0.8867433071136475,nan,nan,8.465081,nan,nan,nan,7.0564466,nan,nan,nan,7.8151436,nan,nan,nan,58595.043,-0.7315406799316406,21.537377013826873,0.07226164,0.507031,-0.5486690402030945,0.886202,nan,nan,nan,nan,nan,nan,nan,nan,-0.7326117157936096,-0.652482807636261,-0.6697278618812561,0,1,nan,nan,nan,-142.22747301583695,46.386958138676235,0.004228246344238351,nan,nan,nan,149.60081545134705,-0.7563512325286865,0.5073366488986605,-1.0924330949783325,3.6330101,nan,nan,nan,80.59491025858249,-53.00087990954968,60.8337664206016
2,4.593351938543022,4.302520367984239,2.4772784265626755,2.8902869079971643,1.4817264317880716,10.016325950622559,0.3380742371082306,-1.1830222606658936,-0.39430123567581177,nan,nan,nan,0.0,20.88471839766448,-0.8867433071136475,nan,nan,9.277008,nan,nan,nan,7.679845,nan,nan,nan,8.512117,nan,nan,nan,68168.14,-0.7315406799316406,22.568708036787722,0.040845014,0.44261116,-0.5486690402030945,0.886202,nan,nan,nan,nan,nan,nan,nan,nan,-0.7326117157936096,-0.652482807636261,-0.6697278618812561,0,1,nan,nan,nan,-142.74596879582657,47.00537149892369,1.6067705879021805,nan,nan,nan,150.2947047268568,-0.7563512325286865,0.4436048636953867,-1.0924330949783325,3.6021755,nan,nan,nan,80.80015343120509,-55.64432414076711,63.67842051361495
3,2.9015173588471916,2.7570225949714815,1.5797617115375588,1.8696858573346729,0.9359733415636101,10.016325950622559,0.3380742371082306,-1.1830222606658936,-0.39430123567581177,nan,nan,nan,0.0,20.86774516164448,-0.8867433071136475,nan,nan,9.901029,nan,nan,nan,8.152237,nan,nan,nan,9.037168,nan,nan,nan,77695.805,-0.7315406799316406,22.369203487990596,0.027340515,0.39295405,-0.5486690402030945,0.886202,nan,nan,nan,nan,nan,nan,nan,nan,-0.7326117157936096,-0.652482807636261,-0.6697278618812561,0,1,nan,nan,nan,-141.3933907874622,47.38119568899527,1.025228330531424,nan,nan,nan,149.1245109176375,-0.7563512325286865,0.39350367992526536,-1.0924330949783325,3.585968,nan,nan,nan,81.7927835288613,-55.95010036400364,62.88779437252636
4,0.707155208444038,0.7543448541479726,0.4182799988714321,0.5913958141395317,0.2281145833690445,10.016325950622559,0.3380742371082306,-1.1830222606658936,-0.39430123567581177,nan,nan,nan,0.0,20.870514615819665,-0.8867433071136475,nan,nan,5.201803,nan,nan,nan,4.5038996,nan,nan,nan,4.9383516,nan,nan,nan,30338.236,-0.7315406799316406,21.756093059289544,0.8118422,0.7637918,-0.5486690402030945,0.886202,nan,nan,nan,nan,nan,nan,nan,nan,-0.7326117157936096,-0.652482807636

Please refer to [`vaex`'s documentation](https://vaex.io/docs/tutorial.html) for further help on how to use `vaex` dataframes: the following line for example isolate only the rows with non-NaN photometry.

In [8]:
survey[~survey.gaiadr2_gmag.isna()]

#,A_0,A_gaiadr2_g_bpmag,A_gaiadr2_g_rpmag,A_gaiadr2_gmag,E(B-V),age,alpha,calcium,carbon,dec,dec_Err,dec_Sig,dform,dmod,feh,gaiadr2_g_bpmag,gaiadr2_g_bpmag_Err,gaiadr2_g_bpmag_Intrinsic,gaiadr2_g_bpmag_Sig,gaiadr2_g_rpmag,gaiadr2_g_rpmag_Err,gaiadr2_g_rpmag_Intrinsic,gaiadr2_g_rpmag_Sig,gaiadr2_gmag,gaiadr2_gmag_Err,gaiadr2_gmag_Intrinsic,gaiadr2_gmag_Sig,glat,glon,grav,helium,log10_NH,lum,mact,magnesium,mtip,mub,mudec,mudec_Err,mudec_Sig,mul,mura,mura_Err,mura_Sig,neon,nitrogen,oxygen,parentid,partid,pi,pi_Err,pi_Sig,px,py,pz,ra,ra_Err,ra_Sig,rad,silicon,smass,sulphur,teff,vr,vr_Err,vr_Sig,vx,vy,vz
0,0.5968695273372091,0.6134133556012421,0.34736789443008625,0.46617317284163673,0.19253855720555133,9.541040420532227,-0.12732818722724915,-0.9319211840629578,-1.1354221105575562,23.517656,-6.046303323376733e-07,5.2571426e-07,0.0,20.84754502003177,-0.8923100233078003,21.396175,-0.17414838516711492,0.109366186,0.4227643,20.388311,0.19883259760421385,-0.99289364,0.4227643,20.98313,0.034675845574094984,-0.36526373,0.017036235,-20.265621,171.08704,143.3362,-0.83549964427948,21.682457721762407,117.38007,1.242,-1.019638180732727,1.2443578,3.513172188479267,3.4756433158643536,3.427204736212419,2.8578458,-1.4504939005943929,1.5382505144926077,1.5751808532822085,2.8578458,-0.7911292314529419,-0.6384884715080261,-0.75995934009552,35,1,0.39720688252231606,0.3904384049219729,1.8925712,-136.9243352067261,21.473499382256776,-51.17440237294062,62.51311,7.243909296036054e-08,5.2571426e-07,147.74371122234376,0.05995115637779236,1.2457935070239523,-0.40517234802246094,3.685448,nan,nan,nan,-4.794804640370714,43.66729503957819,0.798326513834674
1,0.03240183224912084,0.03414993547059877,0.019130807760238122,0.026667334626947797,0.010452203951329302,8.97172737121582,-0.6125200986862183,-0.12460324913263321,-0.36052706837654114,41.480618,4.365875860144255e-07,3.6566976e-07,0.0,20.93897257012393,-0.5348672270774841,20.983871,-0.016208278154500884,0.026959075,0.32201025,20.511648,0.4686094145012974,-0.9150356,0.32201025,20.607824,0.003854659549329624,-0.36166972,0.0140326675,-6.533239,159.05208,369.8372,-0.6886086463928223,20.417147884093257,108.46986,2.1231031,-1.1473873853683472,1.9929105,-2.102441136818007,-2.4309630222975875,-2.3336165790372227,2.0830476,1.3281715071577405,-0.524134667850378,-0.4227110779422327,2.0830476,-0.44875627756118774,-1.4166113138198853,-0.40134233236312866,138,1,-0.7468680151846264,-0.7533574292532406,1.3164111,-142.97762453528603,54.73491449375926,-17.533098219052395,64.27824,1.9834504127347728e-07,3.6566976e-07,154.0971171552255,-0.4361392557621002,2.123614686399606,-1.2804145812988281,3.7215486,nan,nan,nan,-1.8718466960439488,4.028888606484042,-103.68306654562274
2,0.1367282662724903,0.13681457303676017,0.07814074535173385,0.10068780081568864,0.04410589234596461,10.069992065429688,0.5896989703178406,-0.9664415121078491,-0.9518471360206604,37.23763,1.8628932298729322e-07,2.1994336e-07,0.0,21.773146184828576,-0.801622748374939,20.979319,0.10798543964951746,-1.0386255,0.21877547,19.17353,-0.1146483067403298,-2.5623941,0.21877547,20.118101,0.004118068925999436,-1.7598503,0.010660492,-0.99418694,169.38763,10.833822,-0.32748517394065857,21.04243662177693,558.4033,0.8450456,-0.21192379295825958,0.86665446,-0.4985759766664853,-2.4550694541417575,-2.474184581515551,1.3387846,2.6692705848408944,1.160266054548476,1.1128577418471601,1.3387846,-0.3413775563240051,-0.13905183970928192,-0.8814153671264648,212,1,-1.1239242183866573,-1.1283436942160274,0.7917961,-222.36747086527308,41.66461293569632,-3.9260301926370644,78.40865,-7.185046775275613e-08,2.1994336e-07,226.2711775352154,-0.7138696312904358,0.8677262727803007,-1.077961802482605,3.6162643,nan,nan,nan,-51.41373916074214,-3.782666245772003,52.36346019742211
3,0.0276760646802661,0.028464631949460837,0.01613728310687536,0.021720398372794374,0.008927762800085838,9.613767623901367,-0.1432933509349823,-1.0196176767349243,-0.6505561470985413,27.43915,2.0303131545505185e-07,5.2313027e-07,0